pca_tfidf_200features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Tfidf/tfidf_withstopwordchangelevels_200_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,0,-0.906072,-0.676420,-0.175982,-0.049499,-0.468110,0.959464,1.048553,-0.228343,0.019280,...,-0.057293,0.855834,-0.408111,-3.007707,0.471259,-1.609153,-2.661308,-0.483417,4.128491,-0.134168
1,1,-1.068756,-0.832233,-0.215351,-0.222507,2.326828,1.088097,-1.571020,-0.430585,-1.578206,...,0.710536,-0.348293,0.017130,0.115203,0.381732,0.357276,-0.040838,0.424234,-0.448003,0.248756
2,2,-0.572574,-0.103528,-0.214503,-0.206482,-0.595500,-0.031554,0.148982,-0.035070,-0.033863,...,-0.346431,-0.276536,0.316045,-0.251992,-0.406655,0.509367,0.212690,0.151947,0.157507,0.028765
3,3,-0.833341,-0.554500,-0.145166,-0.084242,-0.939621,-0.082813,0.231370,-0.228585,-0.145625,...,0.479135,-0.066320,-0.321325,-0.418405,-0.308807,0.591715,-0.134798,1.276892,0.743146,-0.766192
4,4,-0.884707,-0.619415,-0.160572,0.013895,-0.332685,0.404581,0.380738,-0.275527,-0.127615,...,2.421403,0.666561,0.558502,-0.713223,-0.814704,-0.319202,0.634304,1.007770,0.860486,-0.235200


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.906072,-0.676420,-0.175982,-0.049499,-0.468110,0.959464,1.048553,-0.228343,0.019280,-0.027009,...,-0.057293,0.855834,-0.408111,-3.007707,0.471259,-1.609153,-2.661308,-0.483417,4.128491,-0.134168
1,-1.068756,-0.832233,-0.215351,-0.222507,2.326828,1.088097,-1.571020,-0.430585,-1.578206,0.224439,...,0.710536,-0.348293,0.017130,0.115203,0.381732,0.357276,-0.040838,0.424234,-0.448003,0.248756
2,-0.572574,-0.103528,-0.214503,-0.206482,-0.595500,-0.031554,0.148982,-0.035070,-0.033863,0.085805,...,-0.346431,-0.276536,0.316045,-0.251992,-0.406655,0.509367,0.212690,0.151947,0.157507,0.028765
3,-0.833341,-0.554500,-0.145166,-0.084242,-0.939621,-0.082813,0.231370,-0.228585,-0.145625,0.086207,...,0.479135,-0.066320,-0.321325,-0.418405,-0.308807,0.591715,-0.134798,1.276892,0.743146,-0.766192
4,-0.884707,-0.619415,-0.160572,0.013895,-0.332685,0.404581,0.380738,-0.275527,-0.127615,0.123550,...,2.421403,0.666561,0.558502,-0.713223,-0.814704,-0.319202,0.634304,1.007770,0.860486,-0.235200


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 200), (524, 200))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [16]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [17]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3479636775438302


**Decision Tree**

In [18]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [19]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [20]:

y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [21]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [22]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [23]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.52541,0.52541,0.519156,0.504894,0.427481,0.427481,0.409822,0.401013


**Grid Search -- Decision Tree**

In [ ]:
clf_dt_gs = DecisionTreeClassifier()
parameters = {'criterion': ['entropy','gini'], 
              'max_depth': [3,4,5],
              'min_samples_split': [5,10],
              'min_samples_leaf': [2]}

from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(clf_dt_gs, parameters, cv=5, scoring='recall', verbose=50, n_jobs=-1, refit=True)

In [ ]:
gs.fit(X_train, y_train )

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [3, 4, 5], 'min_samples_leaf': [2],
                         'min_samples_split': [5, 10]},
             scoring='recall', verbose=50)

In [ ]:
gs.best_params_

{'criterion': 'entropy',
 'max_depth': 3,
 'min_samples_leaf': 2,
 'min_samples_split': 5}

In [ ]:
train_pred_dt = gs.predict(X_train)
test_pred_dt = gs.predict(X_test)

In [ ]:
def evaluate_model(act, pred):
    print("Confusion Matrix \n", confusion_matrix(act, pred))
    print("Accuracy : ", accuracy_score(act, pred))
    print("Recall   : ", recall_score(act, pred,average="weighted"))
    print("Precision: ", precision_score(act, pred,average="weighted"))
    print("F1_score : ", f1_score(act, pred,average="weighted"))

In [ ]:
print("--Train--")
evaluate_model(y_train, train_pred_dt)

--Train--
Confusion Matrix 
 [[135 226   0]
 [117 436   0]
 [110 196   0]]
Accuracy :  0.4680327868852459
Recall   :  0.4680327868852459
Precision:  0.3406874810121799
F1_score :  0.3906295588930706


In [ ]:
print("--Test--")
evaluate_model(y_test, test_pred_dt)

--Test--
Confusion Matrix 
 [[ 48 109   0]
 [ 61 176   0]
 [ 38  92   0]]
Accuracy :  0.42748091603053434
Recall   :  0.42748091603053434
Precision:  0.30898323652055837
F1_score :  0.353909645436664


**Random Forest**

In [24]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [25]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [26]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [27]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9917    0.9945    0.9931       361
           2     0.9982    0.9910    0.9946       553
           3     0.9903    1.0000    0.9951       306

    accuracy                         0.9943      1220
   macro avg     0.9934    0.9951    0.9943      1220
weighted avg     0.9943    0.9943    0.9943      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3014    0.1401    0.1913       157
           2     0.4510    0.7764    0.5705       237
           3     0.3023    0.1000    0.1503       130

    accuracy                         0.4179       524
   macro avg     0.3516    0.3388    0.3040       524
weighted avg     0.3693    0.4179    0.3527       524



In [28]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.525410,0.525410,0.519156,0.504894,0.427481,0.427481,0.409822,0.401013
1,RandomForest,0.994262,0.994262,0.994287,0.994262,0.417939,0.417939,0.369274,0.352655


**Grid Search -- Random Forest**

In [ ]:
rfc=RandomForestClassifier(random_state=42)

In [ ]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 500]})

In [ ]:
CV_rfc.best_params_ 

{'criterion': 'entropy',
 'max_depth': 7,
 'max_features': 'log2',
 'n_estimators': 200}

In [ ]:
train_pred_rf = CV_rfc.predict(X_train)
test_pred_rf = CV_rfc.predict(X_test)

In [ ]:
def evaluate_model(act, pred):
    print("Confusion Matrix \n", confusion_matrix(act, pred))
    print("Accuracy : ", accuracy_score(act, pred))
    print("Recall   : ", recall_score(act, pred,average="weighted"))
    print("Precision: ", precision_score(act, pred,average="weighted"))
    print("F1_score : ", f1_score(act, pred,average="weighted"))

In [ ]:
print("--Train--")
evaluate_model(y_train, train_pred_rf)

--Train--
Confusion Matrix 
 [[146 215   0]
 [  0 553   0]
 [  1 207  98]]
Accuracy :  0.6532786885245901
Recall   :  0.6532786885245901
Precision:  0.8017987492601076
F1_score :  0.619863067837584


In [ ]:
print("--Test--")
evaluate_model(y_test, test_pred_rf)

--Test--
Confusion Matrix 
 [[  2 155   0]
 [  5 229   3]
 [  0 127   3]]
Accuracy :  0.44656488549618323
Recall   :  0.44656488549618323
Precision:  0.41234071794565363
F1_score :  0.29518995357321015


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:

classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5702    0.3601    0.4414       361
           2     0.5543    0.8029    0.6558       553
           3     0.6283    0.3922    0.4829       306

    accuracy                         0.5689      1220
   macro avg     0.5843    0.5184    0.5267      1220
weighted avg     0.5776    0.5689    0.5490      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3974    0.1975    0.2638       157
           2     0.4802    0.7173    0.5753       237
           3     0.3152    0.2231    0.2613       130

    accuracy                         0.4389       524
   macro avg     0.3976    0.3793    0.3668       524
weighted avg     0.4145    0.4389    0.4041       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.521311,0.521311,0.553264,0.439495,0.417939,0.417939,0.368828,0.327662
1,RandomForest,0.994262,0.994262,0.994280,0.994266,0.414122,0.414122,0.358981,0.350209
2,Adaboost,0.568852,0.568852,0.577554,0.549015,0.438931,0.438931,0.414483,0.404066


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9852    0.9197    0.9513       361
           2     0.9109    0.9982    0.9525       553
           3     0.9892    0.8954    0.9400       306

    accuracy                         0.9492      1220
   macro avg     0.9617    0.9378    0.9479      1220
weighted avg     0.9525    0.9492    0.9490      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2706    0.1465    0.1901       157
           2     0.4560    0.7004    0.5524       237
           3     0.3067    0.1769    0.2244       130

    accuracy                         0.4046       524
   macro avg     0.3444    0.3413    0.3223       524
weighted avg     0.3634    0.4046    0.3625       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.521311,0.521311,0.553264,0.439495,0.417939,0.417939,0.368828,0.327662
1,RandomForest,0.994262,0.994262,0.994280,0.994266,0.414122,0.414122,0.358981,0.350209
2,Adaboost,0.568852,0.568852,0.577554,0.549015,0.438931,0.438931,0.414483,0.404066
3,GBM,0.949180,0.949180,0.952502,0.949018,0.404580,0.404580,0.363419,0.362472


**SVM Model**

In [29]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [30]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [31]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [32]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5276    0.6620    0.5872       361
           2     0.7009    0.5678    0.6274       553
           3     0.6050    0.6307    0.6176       306

    accuracy                         0.6115      1220
   macro avg     0.6112    0.6202    0.6107      1220
weighted avg     0.6256    0.6115    0.6130      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2816    0.3694    0.3196       157
           2     0.4385    0.3460    0.3868       237
           3     0.2443    0.2462    0.2452       130

    accuracy                         0.3282       524
   macro avg     0.3214    0.3205    0.3172       524
weighted avg     0.3433    0.3282    0.3315       524



In [33]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.525410,0.525410,0.519156,0.504894,0.427481,0.427481,0.409822,0.401013
1,RandomForest,0.994262,0.994262,0.994287,0.994262,0.417939,0.417939,0.369274,0.352655
2,SVM,0.611475,0.611475,0.625566,0.613041,0.328244,0.328244,0.343291,0.331523


**Grid Search -- SVM**

In [ ]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.451 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.451 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.455 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.455 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.455 total time=   0.2s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.451 total time=   0.2s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.451 total time=   0.2s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.455 total time=   0.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.455 total time=   0.2s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.455 total time=   0.2s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.451 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=1, gamma=0.01)


In [ ]:
grid_predictions = grid.predict(X_test)
  
# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           1       0.40      0.01      0.02       157
           2       0.45      0.97      0.62       237
           3       0.17      0.01      0.01       130

    accuracy                           0.45       524
   macro avg       0.34      0.33      0.22       524
weighted avg       0.36      0.45      0.29       524



**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4744    0.5125    0.4927       361
           2     0.5928    0.7107    0.6464       553
           3     0.5449    0.2974    0.3848       306

    accuracy                         0.5484      1220
   macro avg     0.5373    0.5068    0.5079      1220
weighted avg     0.5457    0.5484    0.5353      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3174    0.3376    0.3272       157
           2     0.4898    0.6076    0.5424       237
           3     0.2222    0.1077    0.1451       130

    accuracy                         0.4027       524
   macro avg     0.3431    0.3510    0.3382       524
weighted avg     0.3717    0.4027    0.3793       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.521311,0.521311,0.553264,0.439495,0.417939,0.417939,0.368828,0.327662
1,RandomForest,0.994262,0.994262,0.994280,0.994266,0.414122,0.414122,0.358981,0.350209
2,Adaboost,0.568852,0.568852,0.577554,0.549015,0.438931,0.438931,0.414483,0.404066
3,GBM,0.949180,0.949180,0.952502,0.949018,0.404580,0.404580,0.363419,0.362472
4,SVM,0.627869,0.627869,0.672959,0.601646,0.406489,0.406489,0.373168,0.375551
5,KNN,0.548361,0.548361,0.545723,0.535285,0.402672,0.402672,0.371750,0.379326


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3145    0.8920    0.4650       361
           2     0.6138    0.1609    0.2550       553
           3     0.6275    0.1046    0.1793       306

    accuracy                         0.3631      1220
   macro avg     0.5186    0.3858    0.2998      1220
weighted avg     0.5286    0.3631    0.2981      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3142    0.8726    0.4621       157
           2     0.4267    0.1350    0.2051       237
           3     0.3077    0.0308    0.0559       130

    accuracy                         0.3302       524
   macro avg     0.3495    0.3461    0.2410       524
weighted avg     0.3635    0.3302    0.2451       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.521311,0.521311,0.553264,0.439495,0.417939,0.417939,0.368828,0.327662
1,RandomForest,0.994262,0.994262,0.994280,0.994266,0.414122,0.414122,0.358981,0.350209
2,Adaboost,0.568852,0.568852,0.577554,0.549015,0.438931,0.438931,0.414483,0.404066
3,GBM,0.949180,0.949180,0.952502,0.949018,0.404580,0.404580,0.363419,0.362472
4,SVM,0.627869,0.627869,0.672959,0.601646,0.406489,0.406489,0.373168,0.375551
5,KNN,0.548361,0.548361,0.545723,0.535285,0.402672,0.402672,0.371750,0.379326
6,GNB,0.363115,0.363115,0.528644,0.298146,0.330153,0.330153,0.363459,0.245098


Link Grid Search https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/

Link Grid Search https://www.kaggle.com/code/sociopath00/random-forest-using-gridsearchcv